# Assignment: Lab 03 - Computer Vision
## Krishu Wadhwa - euj7fh
## October 09, 2024

In [2]:
# imports 

import requests
import urllib.request
import streamlit as st
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/krishuwadhwa/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


**1. To load the data, run the following code in a chunk:**

In [3]:
from keras.datasets import mnist
df = mnist.load_data('minst.db')
train,test = df
X_train, y_train = train
X_test, y_test = test

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


The `y_test` and `y_train` vectors, for each index `i`, tell you what number is written in the corresponding index in `X_train[i]` and `X_test[i]`. The value of `X_train[i]` and `X_test[i]`, however, is a 28x28 array whose entries contain values between 0 and 256. Each element of the matrix is essentially a "pixel" and the matrix encodes a representation of a number.

To visualize this, run the following code to see the first ten numbers:


In [6]:
np.set_printoptions(edgeitems=30, linewidth=100000)
for i in range(5): 
    print(y_test[i],'\n') # print the label
    print(X_test[i],'\n') # print the matrix of values
    plt.contourf(np.rot90(X_test[i].transpose())) # make a contour plot of the matrix values
    plt.show()

7 

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  84 185 159 151  60  36   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 222 254 254 254 254 241 198 198 198 198 198 198 19

/var/folders/8k/4kz648397lv0fw3qhymwqpk00000gn/T/ipykernel_50347/1106075888.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


OK, those are the data: Labels attached to handwritten digits encoded as a matrix.

**2. What is the shape of `X_train` and `X_test`? What is the shape of `X_train[i]` and `X_test[i]` for each index `i`? What is the shape of `y_train` and `y_test`?**

In [9]:
# shape of x_train and x_test
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


The shapes of `X_train` and `X_test` indicate that there are 60,000 training images and 10,000 test images, and each are of size 28x28 pixels.

In [ ]:
# shape of y_train and y_test
print(y_train.shape)
print(y_test.shape)

The shapes of `y_train` and `y_test` indicate that `y_train` has 60,000 numeral labels and `y_test` has 10,000 numeral labels. Each respectively correspond to a digit in the associated X_train and X_test matrices.

In [10]:
# shape of x_train[i] and x_test[i] for each index i
print(X_train[0].shape)
print(X_test[0].shape)

(28, 28)
(28, 28)


The shapes of `X_train[i]` and `X_test[i]` refer to the individual image matrices of size 28x28 pixels. So, for each image matrix (like X_train[i]), there is a corresponding numeral (like y_train[i]) that tells us which digit the image represents.

**3. Use Numpy's `.reshape()` method to convert the training and testing data from a matrix into a vector of features. So, `X_test[index].reshape((1,784))` will convert the `index`-th element of `X_test` into a `28x28=784`-length row vector of values, rather than a matrix. Turn `X_train` into an `N x 784` matrix `X` that is suitable for scikit-learn's kNN classifier, where `N` is the number of observations and `784=28*28`.**

FileNotFoundError: [Errno 2] No such file or directory: './data/Z_train.parquet'